# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_pd = pd.read_csv("../WeatherPy/weather_pd.csv")

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_pd[["Latitude", "Longitude"]].astype(float)
weights= weather_pd["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather_pd = weather_pd.loc[(weather_pd['Max Temperature']<80)& 
                                  (weather_pd['Max Temperature']>70) &
                                  (weather_pd['Wind Speed']<10) & 
                                  (weather_pd['Cloudiness']==0)]
ideal_weather_pd= ideal_weather_pd.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df= ideal_weather_pd
hotel_df['Hotel Name']=""

In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"location":"",
          "radius": 5000,
          "keyword":"hotel",
          "key": g_key}

for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lng= row['Longitude']
    params['location'] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}")
    response = requests.get(base_url, params=params).json()
     
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")

        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Retrieving Results for Index 214
Closest hotel is The Esplanade Hotel.
Retrieving Results for Index 293
Closest hotel is Haritha Hotel Alampur.
Retrieving Results for Index 335
Closest hotel is Ayers Rock Resort.
Retrieving Results for Index 342
Closest hotel is LocLoc Beachfront.
Retrieving Results for Index 366
Missing field/result... skipping.
Retrieving Results for Index 451
Missing field/result... skipping.
Retrieving Results for Index 496
Closest hotel is Lucania Palazzo Hotel.
Retrieving Results for Index 536
Closest hotel is Hotel 381 Panguipulli.


In [8]:
narrowed_city_df= hotel_df.dropna()

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))